#### Pydantic Basics: Creating and Using Models
Pydantic models are the foundation of data validation in Python. They use Python type annotations to define the structure and validate data at runtime. Here's a detailed exploration of basic model creation with several examples.

In [1]:
from dataclasses import dataclass

In [2]:
@dataclass
class Person():
    name:str
    age:int
    city:str

In [3]:
person=Person("Agam", 34, "Ahmedabad")
print(person)

Person(name='Agam', age=34, city='Ahmedabad')


In [4]:
person=Person(name="Agam", age=34, city=35)
print(person)

Person(name='Agam', age=34, city=35)


In [5]:
## Pydantic to resolve tye check


from pydantic import BaseModel

In [6]:
class Person1(BaseModel):
    name:str
    age:int
    city:str

In [7]:
person=Person1(name="Agam", age=34, city="Ahmedabad")
print(person)

name='Agam' age=34 city='Ahmedabad'


In [8]:
person=Person1(name="Agam", age=34, city=24)
print(person)

ValidationError: 1 validation error for Person1
city
  Input should be a valid string [type=string_type, input_value=24, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

#### 2. Model with Optional Class
Add optional fields using Python's Optional type:

In [10]:
from typing import Optional

class Employee(BaseModel):
    id:int
    name:str
    department:str
    salary:Optional[float]=None     # Optional with default value
    is_active:Optional[bool]=True   # Optional field with default value

In [11]:
emp1 = Employee(id=1, name="John", department="CS")
print(emp1)

id=1 name='John' department='CS' salary=None is_active=True


In [12]:
emp2 = Employee(id=2, name="Krish", department="CS", salary="30000")
print(emp2)

id=2 name='Krish' department='CS' salary=30000.0 is_active=True


Definition:
- Optional[type]: Indicates the field can be None

- Default value (= None or = True): Makes the field optional

- Required fields must still be provided

- Pydantic validates types even for optional fields when values are provided

In [13]:
emp3 = Employee(id=3, name="Krish", department="CS", salary="30000", is_active=1)
print(emp3)

id=3 name='Krish' department='CS' salary=30000.0 is_active=True


In [14]:
from typing import List

class Classroom(BaseModel):
    room_number:str
    students: List[str] #List of strings
    capacity:int

In [15]:
# Create a classroom
classroom = Classroom(
    room_number="A101",
    students=("Alice", "Bob", "Charlie"),
    capacity=30
)
print(classroom)

room_number='A101' students=['Alice', 'Bob', 'Charlie'] capacity=30


In [16]:
list(("Alice", "Bob", "Charlie"))

['Alice', 'Bob', 'Charlie']

In [17]:
# Create a classroom
classroom1 = Classroom(
    room_number="A101",
    students=("Alice", 123, "Charlie"),
    capacity=30
)
print(classroom1)

ValidationError: 1 validation error for Classroom
students.1
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

In [18]:
try:
    invalid_val=Classroom(room_number="A1",students=["Krish",123],capacity=30)
except Exception as e:
    print(e)

1 validation error for Classroom
students.1
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type


#### 4. Model with Nested Models
Create complex structures with nested models:

In [20]:
from pydantic import BaseModel

class Address(BaseModel):
    street:str
    city:str
    zip_code:str

class Customer(BaseModel):
    customer_id:int
    name:str
    address:Address  ## Nested Model

In [21]:
customer=Customer(customer_id=1,name="Krish",
                  address={"street":"Main street","city":"Boston","zip_code":"02108"})

print(customer)

customer_id=1 name='Krish' address=Address(street='Main street', city='Boston', zip_code='02108')


In [22]:
customer=Customer(customer_id=1,name="Krish",
                  address={"street":"Main street","city":123,"zip_code":"02108"})

print(customer)

ValidationError: 1 validation error for Customer
address.city
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

#### Pydantic Fields: Customization and Constraints

The Field function in Pydantic enhances model fields beyond basic type hints by allowing you to specify validation rules, default values, aliases, and more. Here's a comprehensive tutorial with examples.

In [24]:
from pydantic import BaseModel,Field

class Item(BaseModel):
    name:str=Field(min_length=2,max_length=50)
    price:float=Field(gt=0,le=10000)  ## greater than 0 and less than or equal to 1000
    quantity:int=Field(ge=0)

item=Item(name="Book", price=10000,quantity=10)
print(item)

name='Book' price=10000.0 quantity=10


In [25]:
item2=Item(name="Book2", price=100000,quantity=10)
print(item2)

ValidationError: 1 validation error for Item
price
  Input should be less than or equal to 10000 [type=less_than_equal, input_value=100000, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/less_than_equal

In [26]:
class User(BaseModel):
    username:str=Field(description="Unique username for the user")
    age:int=Field(default=18,description="User age default to 18 ")
    email:str= Field(default_factory=lambda: "user@example.com",description="Default email address")


# Examples
user1 = User(username="alice")
print(user1)




username='alice' age=18 email='user@example.com'


In [27]:
user2 = User(username="bob", age=25, email="bob@domain.com")
print(user2)

username='bob' age=25 email='bob@domain.com'


In [28]:
User.model_json_schema()

{'properties': {'username': {'description': 'Unique username for the user',
   'title': 'Username',
   'type': 'string'},
  'age': {'default': 18,
   'description': 'User age default to 18 ',
   'title': 'Age',
   'type': 'integer'},
  'email': {'description': 'Default email address',
   'title': 'Email',
   'type': 'string'}},
 'required': ['username'],
 'title': 'User',
 'type': 'object'}